In [1]:
# W2V两种模式

![jupyter](images/weeks3_img0.png)

In [ ]:
#W2V他的目标是基于当前词预测周边词，或者是基于周边词来预测当前词，本质是“词预测！”，embedding只是他的副产物！！

In [2]:
#  w1, w2, w3, w4, w5, w6, w7, w8
# window = 3，代表我们的上下文是6个词（小细节）
# 如果我们预估的是w5这个词，那么我们要想知道这个词是否出现，我们需要把 w2, w3, w4, w6, w7, w8 这6个词一起作为我们当前这个词w5的上下文
# 也就是说，我们的w5是否要出现，需要 w2, w3, w4, w6, w7, w8 共同决定当前这个词的出现，我们要去遍历我们的整个词库

In [ ]:
# 我   爱    北京    天安门    城楼    上   的    国徽
# w1   w2     w3       w4       w5     w6   w7     w8

In [ ]:
# 以window=3，预估词=w5
# CBOW要求出 p(城楼| context(城楼)) 概率最大
# context（城楼） = {爱    北京    天安门   上   的    国徽}
# 求解的时候p(城楼| context(城楼)) =  p(wt|context(wt)) = max (p(w1|context(wt)), p(w2|context(wt)), ... , p(wn|context(wt))) 
# (w1, w2 , ....., wn就是我们整个词库中的所有的词)

In [ ]:
# 以window=3，当前词=w5
# Skip-gram要求出 p(context(城楼)|城楼) 概率最大
# context（城楼） = {爱    北京    天安门   上   的    国徽}
# 求解的时候 p(context(wt)|wt) = max（P(爱|wt) * P（北京|wt）*P（天安门|wt）*P(上|wt)*P(的|wt)*P(国徽|wt)）
# （P(爱|wt) = max (p(w1|wt), p(w2|wt), ... , p(wn|wt)) 
# (w1, w2 , ....., wn就是我们整个词库中的所有的词)

![jupyter](images/weeks3_img0_0.png)

In [3]:
# CBOW 是 Continuous Bag-of-Words Model 的缩写，是一种根据上下文的词语预测当前词语的出现概率的模型。

![jupyter](images/weeks3_img1.png)

In [4]:
# 从上述描述我们可以知道，softmax遍历我们整个词库去做预测，开销过大，是否有一个非常简洁的方法，能将计算的时间复杂度降下来

![jupyter](images/weeks3_img2.png)

![jupyter](images/weeks3_img3.png)

![jupyter](images/weeks3_img4.png)

![jupyter](images/weeks3_img5.png)

![jupyter](images/weeks3_img6.png)

![jupyter](images/weeks3_img7.png)

![jupyter](images/weeks3_img8.png)

In [5]:
# Skip-gram只是逆转了CBOW的因果关系而已，即已知当前词语，预测上下文

![jupyter](images/weeks3_img9.png)

![jupyter](images/weeks3_img10.png)

![jupyter](images/weeks3_img11.png)

![jupyter](images/weeks3_img12.png)

![jupyter](images/weeks3_img13.png)

In [ ]:
# 这里的pruning和训练过程中的dropout有类似吗？
# droupout本质是防止过拟合，才出现的。类似人，看到一个东西之后，并不能全部记下来，某一些脑链路上遗忘一些东西。
# pruning本质也是为了避免决策树模型的过拟合。

In [6]:
# 负采样
# word2vec是把整个的空间分成了2的8次方个小格子。

![jupyter](images/weeks3_img14_0.png)

In [ ]:
# 假设我现在有15个词，10个是A词，5个是B词，2:1。那此时A就占2**8的2/3这么多小格子，B就占2**8的1/3这么多小格子。
# 类比，假设我又80W的词，是不是P（w|countext(w)）,此时w更可能是一些高频的词。
# 我们可以通过采样把一些低频的词随机丢掉，减少计算。高频词会随机丢掉一部分，低频词也会随机丢掉一部分。如何丢掉又如何保留呢？
# 通过上面的2**8个小格子，随机采样！！【***给模型当训练样本***】！！

In [7]:
# 负采样是不是有损的？那这样的代价是不是就是准确率降低了一些？
# 一定是有损的。

In [8]:
# CBOW是用上下文预测中心词p(w|C)，Skip-gram是用中心词预测上下文p(C|w)，哪种方法更好？其实差别不大，Skip-gram对于低频词学习的更好一些。
    # 原因也很简单，比如同义的两个字：“凉”和“凊“(jìng)。
    # CBOW架构来学习，那“凉”和“凊“是要进行PK的，比如“冬暖夏_"，CBOW更倾向于将词预测为“凉”，因为“凉”的样本更多，
        # 而“凊“的样本被“凉” 给稀释了。
    # Skip-gram架构来学习，“凊“的样本虽然不多，但是不需要和“凉”竞争资源，所以也能学到一些信息。
    # 假设我们在学校上课，有两种模式，大班授课和一对一辅导。班里有两个同学，一个叫“李凉”，一个叫“李凊“。李凉同学性格外向会来事，
        # 上课积极提问，深受广大师生的喜爱。李凊同学性格内向，不主动，甚至有些老师都不认识。
    # 那么在大班授课的情况下，课上提问问题，自然是李凉同学被提问更多一些，那李凉学习到的东西就更多一些，李凊的一些学习机会其实被李凉抢走了。
    # 那如果是一对一辅导，李凉同学积极学习效果自然不会差，但是李凊相比于大班授课也有了更多的学习机会。
    # 大班授课就是CBOW，一对一辅导就是Skip-gram。

In [1]:
# pip install gensim
from gensim.models import word2vec
# 分词之后，空格隔开的，分词器是jieba
sentences=word2vec.Text8Corpus(u'./data/afterSeg.txt')

In [2]:
# model调用
model=word2vec.Word2Vec(sentences,sg=1,vector_size=50,window=5,min_count=2,negative=3,sample=0.001,hs=1,workers=4)

#该步骤也可分解为以下三步（但没必要）：
#model=gensim.model.Word2Vec() 建立一个空的模型对象
#model.build_vocab(sentences) 遍历一次语料库建立词典
#model.train(sentences) 第二次遍历语料库建立神经网络模型

#sg=1是skip—gram算法，对低频词敏感，默认sg=0为CBOW算法
#size是神经网络层数，值太大则会耗内存并使算法计算变慢，一般值取为100到200之间。
#window是句子中当前词与目标词之间的最大距离，3表示在目标词前看3-b个词，后面看b个词（b在0-3之间随机）
#min_count是对词进行过滤，频率小于min-count的单词则会被忽视，默认值为5。
#negative和sample可根据训练结果进行微调，sample表示更高频率的词被随机下采样到所设置的阈值，默认值为1e-3,
#negative: 如果>0,则会采用negativesamping，用于设置多少个noise words
#hs=1表示层级softmax将会被使用，默认hs=0且negative不为0，则负采样将会被选择使用。
#workers是线程数，此参数只有在安装了Cpython后才有效，否则只能使用单核
# vector_size代表维度的大小[50,100,200]
# 【shift+tab能看参数】

In [3]:
model.window

5

In [4]:
model.min_count

2

In [5]:
model.sg

1

In [6]:
#找出某一个词向量最相近的词集合（）
for val in model.wv.similar_by_word(u"酒店", topn=10):
    print(val[0], val[1])

宾馆 0.847201406955719
疗养院 0.8390887975692749
景区 0.8366877436637878
疗养 0.8100847601890564
温德姆 0.808199405670166
住宿 0.8069859743118286
欧尚 0.798992395401001
入住 0.7949517369270325
民宿 0.7926759719848633
云逸 0.7886755466461182


In [41]:
# 我们一般会基于以上方法，来为一批词寻找“同义词/近义词”，word2vec本质上寻找的是具有相同上下文信息的同位词，CBOW、SKIP-GRAM。
# 往往同义词、近义词都有相同的上下文信息。

In [7]:
word1, word2 = '景区', '景点'

In [8]:
# 查看两个词的相似性（cos距离）
model.wv.similarity(word1, word2)

0.75557894

In [9]:
w1 = model.wv.get_vector(word1)

In [10]:
w2 = model.wv.get_vector(word2)

In [11]:
import numpy as np

In [12]:
print(np.dot(w1,w2))

6.012758


In [ ]:
#  归一化上，word2vec返回的embedding默认是没有归一化的

In [13]:
# L2范数
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v/norm

In [14]:
w1, w2 = normalize(w1), normalize(w2)

In [15]:
wz = np.dot(w1,w2)

In [16]:
print(wz)

0.75557894


In [17]:
# 找到最不相关的那些词
word_list = [u'宾馆', u'酒店', u'饭店', u'机场']
model.wv.doesnt_match(word_list)

'机场'

In [7]:
model.wv.get_vector(u'酒店')

array([-0.43223706, -0.7684264 ,  0.04520439, -0.28358904, -0.2617823 ,
       -0.2630291 , -0.12781806, -0.12604815, -0.48844293, -0.2605386 ,
        0.33824894, -0.49821886, -0.00955847,  0.70596665,  0.1510536 ,
       -0.2153847 ,  0.51420397, -0.2103548 ,  0.38993806,  0.18791524,
       -0.14637569,  0.2426362 ,  0.22899625,  0.20810509,  0.37011763,
       -0.4239186 , -0.50217986,  0.27683026, -0.46002465, -0.20205857,
        0.04558575,  0.03964701, -0.14132597, -0.0924281 ,  0.23555522,
       -0.05669046,  0.11358196, -0.8330673 ,  0.5501238 , -0.04760638,
       -0.19001718, -0.24994084, -0.15934081,  0.04314266,  0.2226157 ,
        0.41290075, -0.10798363,  0.20680472,  0.25440896,  0.905935  ],
      dtype=float32)

In [19]:
# 动手实现一个 word2vec
import tensorflow as tf
# tf1 need
tf.enable_eager_execution()
import numpy as np
import matplotlib.pyplot as plt

In [20]:
sentences = []
with open('data/afterSeg.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        sentences.append(line)

In [21]:
sentences[0]

'环境 优雅 ， 服务到位 ， 味道 不错 。 鹅肝 酱配 葱油饼 亮 \n'

In [22]:
sentences = sentences[0: 500]

In [24]:
word_sequence = " ".join(sentences).split()# 用空格分开,所有的单词 有重复单词
word_list = " ".join(sentences).split()
word_list = list(set(word_list)) #自动清除集合类型中的元素重复数据(set),以及元素排序
word_dict = {w: i for i, w in enumerate(word_list)} #编号,单词 : 编号
batch_size = 20  # To show 2 dim embedding graph
embedding_size = 2  # To show 2 dim embedding graph
voc_size = len(word_list)

In [25]:
len(sentences)

500

In [26]:
def random_batch(data, size):
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(data)), size, replace=False)
 
    for i in random_index:
        random_inputs.append(np.eye(voc_size)[data[i][0]])  # target
        random_labels.append(data[i][1])  # context word

    return random_inputs, random_labels

skip_grams = []
for i in range(1, len(word_sequence) - 1):
    target = word_dict[word_sequence[i]]
    context = [word_dict[word_sequence[i - 1]], word_dict[word_sequence[i + 1]]]
 
    for w in context:
        skip_grams.append([target, w])
 
input_batch, target_batch = random_batch(skip_grams, batch_size)
input_batch = np.array(input_batch,dtype=np.float32)
target_batch = np.array(target_batch,dtype=np.float32)

In [27]:
class Word2Vec(tf.keras.Model):
    def __init__(self):
        super(Word2Vec, self).__init__()
 
        self.W = tf.Variable(-2 * np.random.rand(voc_size, embedding_size) + 1, dtype=tf.float32)
        self.WT = tf.Variable(-2 * np.random.rand(embedding_size, voc_size) + 1, dtype=tf.float32)
 
    def call(self, X):
        hidden_layer = tf.matmul(X, self.W)
        output_layer = tf.matmul(hidden_layer, self.WT)
        return output_layer

In [28]:
model = Word2Vec()
# tf2.X need
# optimizer = tf.optimizers.Adam()
# tf1.X need
optimizer = tf.train.AdamOptimizer()
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',
              metrics=['acc'])#记录参数,记录loss,正确率acc
output = model(input_batch)
history = model.fit(input_batch,target_batch,epochs=2000)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 20 samples
Epoch 1/2000
20/20 [==============================] - 0s 5ms/sample - loss: 14.9046 - acc: 0.0000e+00
Epoch 2/2000
20/20 [==============================] - 0s 299us/sample - loss: 14.8917 - acc: 0.0000e+00
Epoch 3/2000
20/20 [==============================] - 0s 399us/sample - loss: 14.8797 - acc: 0.0000e+00
Epoch 4/2000
20/20 [==============================] - 0s 848us/sample - loss: 14.8684 - acc: 0.0000e+00
Epoch 5/2000
20/20 [==============================] - 0s 249us/sample - loss: 14.8577 - acc: 0.0000e+00
Epoch 6/2000
20/20 [==============================] - 0s 349us/sample - loss: 14.8475 - acc: 0.0000e+00
Epoch 7/2000
20/20 [==============================] - 0s 249us/sample - loss: 14.8377 - acc: 0.0000e+00
Epoch 8/2000
20/20 [==============================] - 0s 349us/sample - loss: 14.8284 - acc: 0.0000e+00
Epoch 9/2000
20/20 [==============================] - 0s

Epoch 76/2000
20/20 [==============================] - 0s 299us/sample - loss: 14.0052 - acc: 0.0000e+00
Epoch 77/2000
20/20 [==============================] - 0s 299us/sample - loss: 13.9692 - acc: 0.0000e+00
Epoch 78/2000
20/20 [==============================] - 0s 275us/sample - loss: 13.9452 - acc: 0.0000e+00
Epoch 79/2000
20/20 [==============================] - 0s 399us/sample - loss: 13.9267 - acc: 0.0000e+00
Epoch 80/2000
20/20 [==============================] - 0s 301us/sample - loss: 13.9115 - acc: 0.0000e+00
Epoch 81/2000
20/20 [==============================] - 0s 201us/sample - loss: 13.8984 - acc: 0.0000e+00
Epoch 82/2000
20/20 [==============================] - 0s 401us/sample - loss: 13.8867 - acc: 0.0000e+00
Epoch 83/2000
20/20 [==============================] - 0s 352us/sample - loss: 13.8760 - acc: 0.0000e+00
Epoch 84/2000
20/20 [==============================] - 0s 399us/sample - loss: 13.8662 - acc: 0.0000e+00
Epoch 85/2000
20/20 [==============================] - 

Epoch 154/2000
20/20 [==============================] - 0s 301us/sample - loss: 13.4410 - acc: 0.0000e+00
Epoch 155/2000
20/20 [==============================] - 0s 249us/sample - loss: 13.4352 - acc: 0.0000e+00
Epoch 156/2000
20/20 [==============================] - 0s 249us/sample - loss: 13.4293 - acc: 0.0000e+00
Epoch 157/2000
20/20 [==============================] - 0s 251us/sample - loss: 13.4235 - acc: 0.0000e+00
Epoch 158/2000
20/20 [==============================] - 0s 301us/sample - loss: 13.4176 - acc: 0.0000e+00
Epoch 159/2000
20/20 [==============================] - 0s 276us/sample - loss: 13.4118 - acc: 0.0000e+00
Epoch 160/2000
20/20 [==============================] - 0s 399us/sample - loss: 13.4059 - acc: 0.0500
Epoch 161/2000
20/20 [==============================] - 0s 551us/sample - loss: 13.4000 - acc: 0.0500
Epoch 162/2000
20/20 [==============================] - 0s 951us/sample - loss: 13.3941 - acc: 0.0500
Epoch 163/2000
20/20 [==============================] - 0s

Epoch 235/2000
20/20 [==============================] - 0s 249us/sample - loss: 12.2841 - acc: 0.1500
Epoch 236/2000
20/20 [==============================] - 0s 199us/sample - loss: 12.2759 - acc: 0.1500
Epoch 237/2000
20/20 [==============================] - 0s 251us/sample - loss: 12.2677 - acc: 0.1500
Epoch 238/2000
20/20 [==============================] - 0s 251us/sample - loss: 12.2594 - acc: 0.1500
Epoch 239/2000
20/20 [==============================] - 0s 249us/sample - loss: 12.2510 - acc: 0.1500
Epoch 240/2000
20/20 [==============================] - 0s 348us/sample - loss: 12.2426 - acc: 0.1500
Epoch 241/2000
20/20 [==============================] - 0s 249us/sample - loss: 12.2341 - acc: 0.1500
Epoch 242/2000
20/20 [==============================] - 0s 251us/sample - loss: 12.2256 - acc: 0.1500
Epoch 243/2000
20/20 [==============================] - 0s 251us/sample - loss: 12.2169 - acc: 0.1500
Epoch 244/2000
20/20 [==============================] - 0s 301us/sample - loss: 12

Epoch 316/2000
20/20 [==============================] - 0s 499us/sample - loss: 10.2486 - acc: 0.1500
Epoch 317/2000
20/20 [==============================] - 0s 349us/sample - loss: 10.2349 - acc: 0.1500
Epoch 318/2000
20/20 [==============================] - 0s 349us/sample - loss: 10.2210 - acc: 0.1500
Epoch 319/2000
20/20 [==============================] - 0s 200us/sample - loss: 10.2068 - acc: 0.1500
Epoch 320/2000
20/20 [==============================] - 0s 299us/sample - loss: 10.1921 - acc: 0.1500
Epoch 321/2000
20/20 [==============================] - 0s 249us/sample - loss: 10.1770 - acc: 0.1500
Epoch 322/2000
20/20 [==============================] - 0s 399us/sample - loss: 10.1615 - acc: 0.1500
Epoch 323/2000
20/20 [==============================] - 0s 348us/sample - loss: 10.1456 - acc: 0.1500
Epoch 324/2000
20/20 [==============================] - 0s 449us/sample - loss: 10.1291 - acc: 0.1500
Epoch 325/2000
20/20 [==============================] - 0s 648us/sample - loss: 10

...

Epoch 1620/2000
20/20 [==============================] - 0s 349us/sample - loss: 2.8282 - acc: 0.1500
Epoch 1621/2000
20/20 [==============================] - 0s 325us/sample - loss: 2.8549 - acc: 0.1500
Epoch 1622/2000
20/20 [==============================] - 0s 351us/sample - loss: 2.8637 - acc: 0.1500
Epoch 1623/2000
20/20 [==============================] - 0s 251us/sample - loss: 2.8561 - acc: 0.1500
Epoch 1624/2000
20/20 [==============================] - 0s 325us/sample - loss: 2.8559 - acc: 0.1500
Epoch 1625/2000
20/20 [==============================] - 0s 249us/sample - loss: 2.8353 - acc: 0.1500
Epoch 1626/2000
20/20 [==============================] - 0s 299us/sample - loss: 2.8497 - acc: 0.1500
Epoch 1627/2000
20/20 [==============================] - 0s 299us/sample - loss: 2.8526 - acc: 0.1500
Epoch 1628/2000
20/20 [==============================] - 0s 299us/sample - loss: 2.8389 - acc: 0.1500
Epoch 1629/2000
20/20 [==============================] - 0s 299us/sample - loss: 2

Epoch 1701/2000
20/20 [==============================] - 0s 300us/sample - loss: 2.8242 - acc: 0.1500
Epoch 1702/2000
20/20 [==============================] - 0s 249us/sample - loss: 2.8073 - acc: 0.1500
Epoch 1703/2000
20/20 [==============================] - 0s 299us/sample - loss: 2.8191 - acc: 0.1500
Epoch 1704/2000
20/20 [==============================] - 0s 301us/sample - loss: 2.8136 - acc: 0.1500
Epoch 1705/2000
20/20 [==============================] - 0s 351us/sample - loss: 2.8217 - acc: 0.1500
Epoch 1706/2000
20/20 [==============================] - 0s 249us/sample - loss: 2.8261 - acc: 0.1500
Epoch 1707/2000
20/20 [==============================] - 0s 251us/sample - loss: 2.8158 - acc: 0.1500
Epoch 1708/2000
20/20 [==============================] - 0s 249us/sample - loss: 2.7976 - acc: 0.1500
Epoch 1709/2000
20/20 [==============================] - 0s 252us/sample - loss: 2.8278 - acc: 0.1500
Epoch 1710/2000
20/20 [==============================] - 0s 249us/sample - loss: 2

Epoch 1782/2000
20/20 [==============================] - 0s 352us/sample - loss: 2.8435 - acc: 0.1500
Epoch 1783/2000
20/20 [==============================] - 0s 299us/sample - loss: 2.8436 - acc: 0.1500
Epoch 1784/2000
20/20 [==============================] - 0s 2ms/sample - loss: 2.8275 - acc: 0.1500
Epoch 1785/2000
20/20 [==============================] - 0s 349us/sample - loss: 2.8271 - acc: 0.1500
Epoch 1786/2000
20/20 [==============================] - 0s 249us/sample - loss: 2.8193 - acc: 0.1500
Epoch 1787/2000
20/20 [==============================] - 0s 300us/sample - loss: 2.8313 - acc: 0.1500
Epoch 1788/2000
20/20 [==============================] - 0s 501us/sample - loss: 2.8413 - acc: 0.1500
Epoch 1789/2000
20/20 [==============================] - 0s 249us/sample - loss: 2.8416 - acc: 0.1500
Epoch 1790/2000
20/20 [==============================] - 0s 275us/sample - loss: 2.8472 - acc: 0.1500
Epoch 1791/2000
20/20 [==============================] - 0s 299us/sample - loss: 2.8

Epoch 1863/2000
20/20 [==============================] - 0s 499us/sample - loss: 2.8676 - acc: 0.1500
Epoch 1864/2000
20/20 [==============================] - 0s 299us/sample - loss: 2.8808 - acc: 0.1500
Epoch 1865/2000
20/20 [==============================] - 0s 849us/sample - loss: 2.8639 - acc: 0.1500
Epoch 1866/2000
20/20 [==============================] - 0s 299us/sample - loss: 2.8217 - acc: 0.1500
Epoch 1867/2000
20/20 [==============================] - 0s 249us/sample - loss: 2.8216 - acc: 0.1500
Epoch 1868/2000
20/20 [==============================] - 0s 301us/sample - loss: 2.8201 - acc: 0.1500
Epoch 1869/2000
20/20 [==============================] - 0s 299us/sample - loss: 2.8413 - acc: 0.1500
Epoch 1870/2000
20/20 [==============================] - 0s 252us/sample - loss: 2.8535 - acc: 0.1500
Epoch 1871/2000
20/20 [==============================] - 0s 249us/sample - loss: 2.8541 - acc: 0.1500
Epoch 1872/2000
20/20 [==============================] - 0s 299us/sample - loss: 2

Epoch 1944/2000
20/20 [==============================] - 0s 251us/sample - loss: 2.7979 - acc: 0.1500
Epoch 1945/2000
20/20 [==============================] - 0s 249us/sample - loss: 2.8036 - acc: 0.1500
Epoch 1946/2000
20/20 [==============================] - 0s 251us/sample - loss: 2.7990 - acc: 0.1500
Epoch 1947/2000
20/20 [==============================] - 0s 199us/sample - loss: 2.7765 - acc: 0.2000
Epoch 1948/2000
20/20 [==============================] - 0s 200us/sample - loss: 2.7776 - acc: 0.1500
Epoch 1949/2000
20/20 [==============================] - 0s 249us/sample - loss: 2.7831 - acc: 0.1500
Epoch 1950/2000
20/20 [==============================] - 0s 226us/sample - loss: 2.7705 - acc: 0.1500
Epoch 1951/2000
20/20 [==============================] - 0s 451us/sample - loss: 2.7716 - acc: 0.1500
Epoch 1952/2000
20/20 [==============================] - 0s 401us/sample - loss: 2.7752 - acc: 0.1500
Epoch 1953/2000
20/20 [==============================] - 0s 249us/sample - loss: 2

In [ ]:
# optimizer = tf.train.AdamOptimizer()什么含义
# 是一个梯度下降方法，直观感受可以看week2的作业，从0-1实现开平方，里面有给三种方法。

In [ ]:
# 这个100是embedding-size由我们自定义嘛
# 是的，我们可以定义word2vec到任何维度，一般呢50-200维就足够了。
# 也要看数据的大小，千万级数据，每条数据平均长度10-500个词，150维也够了（经验值，实际工作中，你需要额外的训练不同维度的embedding，看每次的效果）。
# 看每次的效果--> emb当成神经网络模型的一维特征来用的，有这个特征且维度=50维，模型训练(deepfm)有个AUC，有这个特征且维度=100维，还有个AUC，看两次AUC的值。
# 30-50个词，emb设置成50维。
# 一个词，你用多少个0-1之间的小数来表示它，就是维度的问题，[0.1, 0.11]，此时就是2维。[0.1, 0.11, 0.212]，此时就是3维度。

In [ ]:
# 工业上基本都掉包吧？还是基本自己实现
# 工业上直接掉包，from gensim.models import word2vec

In [ ]:
# 老师还是刚刚那个问题 归一化哪块 默认是没有做归一化的，但是为什么dot 要做归一化才和默认没有归一化的一样
# 第一呢，word2vec的model.wv.similarity是内部封装了归一化的，所以计算结果是正确的。
# 第二呢，我们在获取word2vec emb的时候，这个emb是没有归一化的，此时我们做np.dot()相乘就错了。
# 怎么办呢？对word2vec的emb，手动归一化（L2范数 normalize），然后在执行np.dot()。
# 为什么要说这个问题呢？DSSM这个模型，这个是推荐Rank线上的emb召回。A/B两个emb相似度的时候，如果用了类似W2V的思想（DSSM也没做归一化），
# DSSM召回结果可能不对。

In [37]:
# 思考题（任务）
# 神经网络如何避免过拟合问题？如何避免欠拟合问题。--面试常见题
# 什么叫过拟合，什么叫欠拟合。

In [56]:
# 上述我们得到了词的embedding的一个描述，但是我们的embedding向量，是否能做推广，让更多的像推荐算法能够适配到
# 我们想得到 uid embedding 或者 item embedding
# graph embedding的概念，用的算法就是skip-gram or cbow。
# 因为我们在推荐系统里，很多内容都是希望embedding学习的更充分，所以我们引入了skip-gram ，希望能够把低频词学的更好一些
# word2vec 得到的词向量的时候，他们输入的文本信息，是由一定规律的，每一词应该和哪些词共现，是有一定的规律的，遵循的是词法规律。
# 相关的用户行为序列，按道理也应该具有一定的规律，这个规律其实就是用户在进行内容点击的时候，肯定倾向于去点那些相关的内容。
# eg：一个球迷，大概率它的行为序列里面都是和足球相关
# 我们得到的行为序列，都是item的一系列的表征，我们只需要引入skip gram 针对上述的序列进行训练，就能得到item的embedding的信息

In [57]:
# w2v做任务不相关语料库 无监督预训练的embedding，直接用在个人任务上，一般能取得效果么？
    # 如果个人任务很个性化。如果预训练语料库不是包含一切，想想微博的语料和人民日报的语料，训练得到的词向量是不一样的。
    # 在偏个性化的推荐系统里面，一定要有一个常识，每个个体都有自己的兴趣，都可以去构建自己的行为序列，但是很多的样本彼此之间就是有一定的关联
    # 足球迷，大概率点击的视频或者文章，就是那些，虽然他们之间有可能出现的顺序不一样，但是他们都是在同一个上下文里面出现的，就好比是很多的文章
    # 文章是由词组成的，不同的语句肯定有不同的成词的方法，但是彼此的搭配信息是在一个上下文中的，这种其实也算是一定的个性化
    # 如果这种方式，希望能得到更多的适配，无非，我们可以引入用户的行为特征 + 内容的特征，去做某一个特定的任务，然后作用在我们的线上，我们直接去
    # 拿线上用户的embedding直接来进行适配，就能实现所谓的个性化的embedding

In [58]:
# fintune和增量学习，形式上差异在哪？都是新加入样本，调参~ 
# fine-tune可以想想下bert的fine-tune，最开始有个基础模型参数。他是用很多数据集训练的，不包含任何的个性化信息。
    # 我们在此基础上fine-tune，生成自己的个性化model。
# 增量学习，就是在CTR预估或者排序模型中，我用最近15天的样本训练了一个model，但是随着时间的推移，用户的兴趣是会改变的，此时model需要更新
    # 如何更新呢？讲一天的数据融合到过去15天的model中，此时是一个增量学习过程。
    # 而不是每次用最新15天的样本从新训练model（效率低，成本高，收益不明显）。

In [60]:
# 在推荐中word2vec的序列长度一般怎么选，怎么获取长短期兴趣
    # 序列的长度，无所谓，因为一篇文章输入我们的系统，我们需要拿到对应的词的向量，文章的长度也是有长，有短的。
    # 比如一个session中，有的用户输入了10个词，有的用户可能输入了30个词，此时两个session是不等长的。最后session embedding = 每个词的average。
    # 序列的长度，首先肯定有作弊的流量（反扒与反反扒的机制），干掉作弊流量。同一个session，会有成千上万的id序列，我们做的时候，以session划分
    # 例如我们的session是10分钟以内，所有点击过的内容信息
# 平衡长短期兴趣，其实就是引入时间的衰减因子，离当前时间近的，尽量保留，远的开始衰减，借助牛顿冷却定理

In [61]:
# 面试的时候 word2vec问的多吗？
    # 它是embedding的鼻祖
    # 在Graph Embedding里面，用到了大量的word2vec方法
# 我们怎么去求每个内容的item的embedding
# 每个用户都有可能观看的一堆的item的list
# item1, item2, item3, ......
# item2, item3, .......
# 将用户的行为序列抽象成我们的文本的sentence信息，也就是句子信息，其中的item我们抽象成为了词的概念
# 我们就可以采用word2vec的方法，将item的embedding得到

In [62]:
# cbow和skip-gram里，从projection层到输出层的网络权重，是最终要的词向量？那从输入层到projection层的网络权重，不可以看成要的词向量么？
    # cbow和skip gram 的词向量是从输入层到projection的权重，projection 到 输出层 的权重也可以描述，但是一般用的是前面那一个

In [63]:
# word2vec里使用2gram，主要处理的地方是在样本准备过程么？ 
# word2vec 我们需要有一个窗口去进行约束，约束的范围其实就是我们的window，window我们真正描绘的是我们的上下文
# window不会超过5，通过两两组合的方式，看重的更多是共现
# 3 gram，相当于进行了三个词的两两共现，但是我们需要的知道的是，我们的context(w1, w2, w3, w4, w5) -> 不就是在模拟，只不过计算的时候用的是共现
# 但是最终我们可以是要把两两共现相加，最后其实描述的就是当前这个词的上下文

In [64]:
# skipgram一定效果比cbow好么？原因除了样本使用率高，还有别的么？
    # 未必，skip gram有它适用范围（低频友好）
    # cbow也有它的适用范围（高频友好）

In [65]:
# word2vec能处理oov么？中文和英文处理oov，能分别举个例子么？
    # word2vec是无法处理OOV的，word2vec要求词是要有共现的，如果这个词都没有，那么就代表对应的语料里面也没有，p(w) = 0 ，向量怎么能学习出来呢？
# 一种方式
    # 会有一个默认的向量（肯定不准）
    # 把词和字一起训练 ，因为词是由字得到的，字与字拼接得到的就是词向量
    # 英文（前缀，后缀， ing， ex,），会把对应的单词拆成subword的方式，去学subword 的pattern的描述，最后对这些pattern去做一个组合，其实和中文
    # 里面对应的字组成词的做法一致
    # bpe, wordpiece （预训练模型里面很常见）
    # 汉字都是由偏旁部首的，我们可以把一个字拆出偏旁部首，学习偏旁部首的embedding ,最后做一个组合

In [66]:
# oov是什么问题？第一次听说，老师能否讲解下
    # out of vocabulary，就是说，一个词，已经超出了我们的词表，训练的时候，没有见过这个词，但是在最终我们适用的时候，出现了这个词
    # 这词在我们的实际embedding中是没有的，就出现了OOV

In [67]:
# 是不是现在业内开始用 Transformers 多了，lstm用的少了
    # transformer 自从 attention is all your need之后，已经称为了nlp界一个非常重要的技术，这个技术的适用范围很广
    # bert （transformer-based的model），计算速度要比lstm要快，基于上下文的方式来解决，确实解决了lstm之前模型无法并行，上下文的语义信息没有
    # transformer玩的更好，lstm还是有它的适用范围（时间序列预测），在一些其他任务上lstm并没有很怂，因为需要大量的资源来适配足够的数据
    # lstm来试一试？EMLO来做的，后来大家都有追风习惯，都去用transoformer

In [68]:
# LR与embedding没有相关性，LR是一个打分器，embedding是一个特征抽取器
# wide & deep，就是把LR的记忆性+神经网络的泛化性进行了一个综合的适配